![graph](https://i.imgur.com/wOyZMnC.png)

In [1]:
from sympy import Matrix, frac

def simplex_method(A, b, c, basis):
    global deltas
    m, n = A.shape

    # compute delta_k
    deltas = [
        c[j] - sum(c[basis[i]]*A[i, j] for i in range(m))
        for j in range(n)
    ]
    
    # check if optimal vertex is reached
    if all(delta >= 0 for delta in deltas):
        xstar = [0] * n
        for i in range(m):
            xstar[basis[i]] = b[i]
        return xstar

    k = deltas.index(min(deltas))  # determine in_index

    # check if target function is unbounded
    if all(A[i, k] <= 0 for i in range(m)): return

    # compute theta_k
    thetas = [
        b[i]/A[i, k] if A[i, k] > 0 else float('inf')
        for i in range(m)
    ]

    l = thetas.index(min(thetas))  # determine out_index
    basis[l] = k                   # change basis

    # perform gauss transformations
    b[l] /= A[l, k]
    A[l, :] /= A[l, k]
    for i in range(m):
        if i == l: continue
        b[i] -= b[l] * A[i, k]
        A[i, :] -= A[l, :] * A[i, k]

    return simplex_method(A, b, c, basis)   # start over

In [8]:
A = Matrix([
    [2,  3, 1, 0,  0],
    [3, -5, 0, 1,  0],
    [5,  3, 0, 0, -1],
])

b = Matrix([19, 17, 17])

c = Matrix([-5, -1, 0, 0, 0])


# M-method
m, n = A.shape
M = max(max(abs(A)), max(abs(b)), max(abs(c))) + 1
basis = []
ys = 0

for i in range(m):
    if A[i, n-m+i] == 1:
        basis.append(n-m+i)
    else:
        col = [0] * m
        col[i] = 1
        A = A.row_join(Matrix(col))
        c = c.col_join(Matrix([M]))
        basis.append(n + ys)
        ys += 1

xstar = simplex_method(A, b, c, basis)

if xstar is None or any(x > 0 for x in xstar[n:]):
    print("No solution")

print('x* =', xstar)
print('L* =', -sum(a*b for a, b in zip(xstar, c)))

x* = [146/19, 23/19, 0, 0, 476/19, 0]
L* = 753/19


In [3]:
def gomory_cut_1(A, b, c, x, basis):
    if all(type(beta) is int or beta.is_integer for beta in x): return x

    global deltas
    l = -1
    for i, beta_i in enumerate(x):
        if not beta_i.is_integer:
            l = i
            break
            
    index = basis.index(l)
    
    new_row = [-frac(alpha) for alpha in A[index, :]]
    new_row.append(1)
    
    m, n = A.shape
    
    b = b.col_join(Matrix([-frac(x[l])]))
    c = c.col_join(Matrix([0]))
    A = A.row_join(Matrix([0]*m))
    A = A.col_join(Matrix([new_row]))

    deltas = [
        -delta/alpha if alpha < 0 else float('inf') 
        for delta, alpha in zip(deltas, new_row)
    ]
    k = deltas.index(min(deltas)) # in_index
    
    basis.append(k)
    l = m # out_index

    b[l] /= A[l, k]
    A[l, :] /= A[l, k]
    for i in range(m):
        if i == l: continue
        b[i] -= b[l] * A[i, k]
        A[i, :] -= A[l, :] * A[i, k]

    x = simplex_method(A, b, c, basis)
    
    return gomory_cut_1(A, b, c, x, basis)

In [4]:
xstar = gomory_cut_1(A[:, :n], b, Matrix(c[:n]), xstar[:n], basis)
print('x* =', xstar)
print('L* =', -sum(a*b for a, b in zip(xstar, c)))

x* = [7, 1, 2, 1, 21, 0, 0]
L* = 36


In [6]:
def gomory_cut_2(A, b, c, x, basis, p):
    if all(type(beta) is int or beta.is_integer for beta in x[:p]): return x

    global deltas
    l = -1
    for i, beta_i in enumerate(x):
        if type(beta_i) is not int and not beta_i.is_integer:
            l = i
            break
            
    index = basis.index(l)
    
    def gamma(j, alpha, beta):
        if j < p:
            if frac(alpha) <= frac(beta):
                return frac(alpha)
            else:
                return frac(beta)*(1 - frac(alpha)) / (1 - frac(beta))
        else:
            if alpha >= 0:
                return alpha
            else:
                return - frac(beta)*alpha / (1 - frac(beta))
        
    
    new_row = [-gamma(j, alpha, x[l]) for j, alpha in enumerate(A[index, :])]
    new_row.append(1)
    
    m, n = A.shape
    
    b = b.col_join(Matrix([-frac(x[l])]))
    c = c.col_join(Matrix([0]))
    A = A.row_join(Matrix([0]*m))
    A = A.col_join(Matrix([new_row]))

    deltas = [
        -delta/alpha if alpha < 0 else float('inf') 
        for delta, alpha in zip(deltas, new_row)
    ]
    k = deltas.index(min(deltas)) # in_index
        
    basis.append(k)
    l = m # out_index
    b[l] /= A[l, k]
    A[l, :] /= A[l, k]
    for i in range(m):
        if i == l: continue
        b[i] -= b[l] * A[i, k]
        A[i, :] -= A[l, :] * A[i, k]

    x = simplex_method(A, b, c, basis)
    return gomory_cut_2(A, b, c, x, basis, p)

In [7]:
xstar = gomory_cut_2(A[:, :n], b, Matrix(c[:n]), xstar[:n], basis, p=1)
print('x* =', xstar)
print('L* =', -sum(a*b for a, b in zip(xstar, c)))

x* = [7, 5/3, 0, 13/3, 23, 0]
L* = 110/3


In [9]:
def dalton_llevelyn(A, b, c, x, basis, discrete_sets):
    if all(beta in s for beta, s in zip(x, discrete_sets)): return x
    
    global deltas
    l = -1
    for i, (beta_i, discrete_set) in enumerate(zip(x, discrete_sets)):
        if beta_i not in discrete_set:
            l = i
            for j in range(len(discrete_set)-1):
                if discrete_set[j] < beta_i < discrete_set[j+1]:
                    left, right = discrete_set[j:j+2]
                    break
            else:
                raise Exception('Additional constraints required')
            break
            
    index = basis.index(l)
    
    def gamma(j, alpha, beta):
        if j in basis: return 0
        if alpha >= 0:
            return alpha
        else:
            return (beta - left)/(right - beta) * (-alpha)
    
    new_row = [-gamma(j, alpha, x[l]) for j, alpha in enumerate(A[index, :])]
    new_row.append(1)
    
    m, n = A.shape

    b = b.col_join(Matrix([-x[l] + left]))
    c = c.col_join(Matrix([0]))
    A = A.row_join(Matrix([0]*m))
    A = A.col_join(Matrix([new_row]))

    deltas = [
        -delta/alpha if alpha < 0 else float('inf') 
        for delta, alpha in zip(deltas, new_row)
    ]
    k = deltas.index(min(deltas)) # in_index
        
    basis.append(k)
    l = m # out_index
    b[l] /= A[l, k]
    A[l, :] /= A[l, k]
    for i in range(m):
        if i == l: continue
        b[i] -= b[l] * A[i, k]
        A[i, :] -= A[l, :] * A[i, k]

    x = simplex_method(A, b, c, basis)
    return dalton_llevelyn(A, b, c, x, basis, discrete_sets)

In [10]:
discrete_sets = [
    [0, 1, 5, 8],
    [0, 2, 3, 7]
]

xstar = dalton_llevelyn(A[:, :n], b, Matrix(c[:n]), xstar[:n], basis, discrete_sets)
print('x* =', xstar)
print('L* =', -sum(a*b for a, b in zip(xstar, c)))

x* = [5, 3, 0, 17, 17, 0]
L* = 28
